# Data Assessment & Analytics

In [1]:
#Needed Libararies

# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# visulizaiton
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geoplot as gplt # for plotting maps

# API retrieval
import requests
import json

# Cleanup
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

## Water Right Data

In [2]:
# #Setting work directory
# workingDir = "G:/Shared drives/WaDE Data/Idaho/WaterAllocation/ProcessedInputData"
# os.chdir(workingDir)
# print("Current working directory: {0}".format(workingDir))

Current working directory: G:/Shared drives/WaDE Data/Idaho/WaterAllocation/ProcessedInputData


## Aggregated Amounts Data
- timeseries data

In [ ]:
#Setting work directory
workingDir = "G:/Shared drives/WaDE Data/Colorado/AggregatedAmounts/ProcessedInputData"
os.chdir(workingDir)
print("Current working directory: {0}".format(workingDir))

In [ ]:
# fileInput = "aggregatedamounts.csv"
# df = pd.read_csv(fileInput)
# print(len(df))
# df.head()

In [ ]:
# # Plot All
# df = df.sort_values(by=['ReportingUnitUUID', 'VariableSpecificUUID', 'ReportYearCV'])
# fig = px.line(df, x="ReportYearCV", y="Amount", color='ReportingUnitUUID')
# fig.show()

In [ ]:
# # Plot One Site
# dfs1 = df.copy()
# dfs1 = dfs1[dfs1['ReportingUnitUUID'] == 'CAag_RU3'].reset_index(drop=True)

# fig = px.line(dfs1, x="ReportYearCV", y="Amount", color='VariableSpecificUUID')
# fig.show()

In [ ]:
# Plotting Points
# dftemp = dfru2.copy()
# dftemp = dftemp[dftemp['State']=='UT'].reset_index(drop=True) # change state entry here
# dftemp['geometry'] = gpd.GeoSeries.from_wkt(dftemp['geometry'], crs="EPSG:4326")
# gdftemp = gpd.GeoDataFrame(dftemp, geometry=dftemp['geometry'], crs="EPSG:4326")
#gplt.pointplot(gdf2)

In [ ]:
fileInput = "reportingunits.csv"
dfru = pd.read_csv(fileInput)
print(len(dfru))
dfru.head()

In [ ]:
#Plotting Polygons
# checking for certain reportingunittypeCV

gdfru = dfru.copy()
#gdfru = gdfru[gdfru['ReportingUnitTypeCV'] == 'County'].reset_index(drop=True)
gdfru['geometry'] = gpd.GeoSeries.from_wkt(gdfru['Geometry'], crs="EPSG:4326")
gdfru = gpd.GeoDataFrame(gdfru, geometry=gdfru['geometry'], crs="EPSG:4326") # covert to geodataframe
gplt.polyplot(gdfru, edgecolor='blue', facecolor='lightgray')

In [ ]:
#gdfru.to_file("checkShapefileNV.shp") # export shape file

## Site Spececific Amounts Data

In [ ]:
#Setting work directory
workingDir = "G:/Shared drives/WaDE Data/Texas/SS_ReservoirsObservationSites/ProcessedInputData"
os.chdir(workingDir)
print("Current working directory: {0}".format(workingDir))

#### variables

In [ ]:
fileInput = "variables.csv"
dfv = pd.read_csv(fileInput)
print(len(dfv))
dfv.head()

#### watersource

In [ ]:
fileInput = "watersources.csv"
dfws = pd.read_csv(fileInput)
print(len(dfws))
dfws.head(1)

#### Sites

In [ ]:
fileInput = "sites.csv"
dfs = pd.read_csv(fileInput)
print(len(dfs))
dfs.head()

In [ ]:
dfs['SiteTypeCV'].unique()

In [ ]:
# Mapping Site.csv Points
gdfs = gpd.GeoDataFrame(dfs, geometry=gpd.points_from_xy(dfs.Longitude.astype(float), dfs.Latitude.astype(float)), crs="EPSG:4326")
gplt.pointplot(gdfs)

#### sitespecificamounts

In [ ]:
fileInput = "sitespecificamounts.csv"
dfsa = pd.read_csv(fileInput)
print(len(dfsa))
dfsa.sort_values(by=['SiteUUID', 'VariableSpecificUUID', "ReportYearCV", "TimeframeStart"])

In [ ]:
dfsa['VariableSpecificUUID'].unique()

In [ ]:
# check for duplicate records
dfsa2 = dfsa.copy()
#df2 = df2.drop_duplicates(subset=['OrganizationUUID', 'SiteUUID', 'VariableSpecificUUID', 'BeneficialUseCategory', 'ReportYearCV'], keep='last')
dfsa2 = dfsa2.drop_duplicates(subset=['OrganizationUUID', 'SiteUUID', 'VariableSpecificUUID', 'BeneficialUseCategory', 'ReportYearCV', 'TimeframeEnd', 'TimeframeStart'], keep='last')
print(len(dfsa2))
dfsa2.head()

In [ ]:
# Plot All
dfsa = dfsa.sort_values(by=['SiteUUID', 'VariableSpecificUUID', 'TimeframeStart'])
fig = px.line(dfsa, x="TimeframeStart", y="Amount", color='SiteUUID')
fig.show()

In [ ]:
# Plot One Site - need to pick a site from above Plot all
dfsa1 = dfsa.copy()
dfsa1 = dfsa1[dfsa1['SiteUUID'] == 'IDssro_S82'].reset_index(drop=True)
fig = px.line(dfsa1, x="TimeframeStart", y="Amount", color='VariableSpecificUUID')
fig.show()

In [ ]:
# Mapping sitespecificamounts.csv Points
fileInput = "sites.csv"
dfs = pd.read_csv(fileInput)
dftemp = pd.merge(df, dfs, on="SiteUUID", how="left")
gdf = gpd.GeoDataFrame(dftemp, geometry=gpd.points_from_xy(dftemp.Longitude.astype(float), dftemp.Latitude.astype(float)), crs="EPSG:4326")
gplt.pointplot(gdf)

In [ ]:
# merge tables.  Export to inspect
dftemp = pd.DataFrame()
dftemp = pd.merge(dfsa, dfv[['VariableSpecificUUID', 'VariableSpecificCV', 'AggregationIntervalUnitCV']], on='VariableSpecificUUID', how='left')
dftemp = pd.merge(dftemp, dfs[['SiteUUID', 'WaterSourceUUIDs', 'SiteName', 'SiteTypeCV']], on='SiteUUID', how='left')
dftemp = pd.merge(dftemp, dfws[['WaterSourceUUID', 'WaterSourceTypeCV']], left_on='WaterSourceUUIDs', right_on='WaterSourceUUID', how='left')
dftemp.to_excel('checkCombined.xlsx', index=False)  # The output.
dftemp.head(1)

In [ ]:
# Plotting Polygons
# dftemp = dfru2.copy()
# dftemp = dftemp[dftemp['State']=='UT'].reset_index(drop=True) # change state entry here
# dftemp['geometry'] = gpd.GeoSeries.from_wkt(dftemp['geometry'], crs="EPSG:4326")
# gdftemp = gpd.GeoDataFrame(dftemp, geometry=dftemp['geometry'], crs="EPSG:4326")
# gplt.polyplot(gdftemp)